In [1]:
#Importing Libraries & Packages
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import os
import zipfile

In [2]:
#Loading in Processed Images
def load_images(dir):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dir,
        image_size=(256, 256),
        batch_size=1,
        label_mode=None
    )
    return dataset.map(lambda x: (x - 128) / 128).repeat()

monet_dataset = load_images('/Users/jschlangen/Desktop/gan-getting-started/monet_jpg')
photo_dataset = load_images('/Users/jschlangen/Desktop/gan-getting-started/photo_jpg')

Found 300 files.
Found 7038 files.
